<a href="https://colab.research.google.com/github/AnitaTasnim/GNN_attack_model/blob/main/nnn_drAlom_provided.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import itertools
import numpy
import networkx as nx
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from tqdm import tqdm
from net import Nettack1 ####### our modified code
from nettackonlyadd import NettackAdd  ######for addition only
import time
import torch
import numpy as np
import torch.nn.functional as F
from deeprobust.graph import utils
import torch.optim as optim
from deeprobust.graph.defense import GCN
from deeprobust.graph.defense import GAT
#from deeprobust.graph.targeted_attack import Nettack
from nettack_dpr import Nettack
from deeprobust.graph.utils import *
from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm
import random
import math
import pandas as pd
from deeprobust.graph.data import Dataset, Dpr2Pyg, Pyg2Dpr
from numba import jit
from allfnc import get_linearized_weight, normalize_adj, compute_logits, find_all_2hop_neighbors1,select_lists_with_element1


def test1(adj, features, target_node):
    ''' test on GCN '''
    gcn = GCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device=device)
    gcn = gcn.to(device)
    gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
    gcn.eval()
    output = gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    print('Target node probs: {}'.format(probs.detach().cpu().numpy()))
    print(output.argmax(1)[target_node])
    print(labels[target_node])
    #acc_test = accuracy(output[idx_test], labels[idx_test])

    #print("Overall test set results:",
     #     "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()



data = Dataset(root='C:\pythonProject1', name='cora')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

print(data)
print(idx_train)

idx_unlabeled = np.union1d(idx_val, idx_test)

print(idx_unlabeled)
print(len(idx_unlabeled))

node_list = random.sample(list(idx_unlabeled),10)

print(node_list)


degrees = adj.sum(0).A1

print(degrees)
print(degrees[400])